In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


Loading the Dataset

In [2]:

data_train=pd.read_csv('../input/students-drop-out-prediction/train.csv')
data_test=pd.read_csv('../input/students-drop-out-prediction/test.csv')


In [3]:
data_train.shape

(3796, 43)

In [4]:
data_test.shape

(1628, 42)

In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3796 entries, 0 to 3795
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3796 non-null   int64  
 1   v_21    3796 non-null   int64  
 2   v_32    3796 non-null   int64  
 3   v_15    3796 non-null   int64  
 4   v_4     3796 non-null   int64  
 5   v_1     3796 non-null   int64  
 6   v_8     3796 non-null   int64  
 7   v_12    3796 non-null   int64  
 8   v_25    3796 non-null   int64  
 9   v_20    3796 non-null   int64  
 10  v_18    3796 non-null   int64  
 11  v_3     3796 non-null   int64  
 12  v_11    3796 non-null   int64  
 13  v_14    3796 non-null   int64  
 14  v_26    3796 non-null   int64  
 15  v_27    3796 non-null   int64  
 16  v_30    3796 non-null   int64  
 17  v_9     3796 non-null   int64  
 18  v_31    3796 non-null   int64  
 19  v_28    3796 non-null   int64  
 20  v_13    3796 non-null   int64  
 21  v_33    3796 non-null   int64  
 22  

#### Data Preprocessing

In [6]:
X = data_train.drop('label',axis=1)
y = data_train['label']

In [7]:
X.shape

(3796, 42)

In [8]:
data_test.shape

(1628, 42)

In [9]:
X.shape

(3796, 42)

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [11]:
X_new  = StandardScaler().fit_transform(X)
data_test_new = StandardScaler().fit_transform(data_test)

In [12]:
data_test_new.shape

(1628, 42)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_new,y, test_size=0.2,random_state=42)

### Our Scoring Metric is: F1_Score

# Selecting and Training Model

### 1. LogisticRegressionCV

In [15]:
from sklearn.linear_model import LogisticRegressionCV

lrcv = LogisticRegressionCV(cv=10,random_state=42)

In [16]:
lrcv.fit(X_train,y_train)

LogisticRegressionCV(cv=10, random_state=42)

In [17]:
print("F1_Score - LogisticRegressionCV: " ,lrcv.score(X_new,y))

F1_Score - LogisticRegressionCV:  0.7426238145416227


### 2. Ridge Classifier CV

In [18]:
from sklearn.linear_model import RidgeClassifierCV

In [19]:
ridge = RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1],cv=10).fit(X_train, y_train)

In [20]:
print("F1_Score -> RidgeClassifierCV: " ,ridge.score(X_new,y))

F1_Score -> RidgeClassifierCV:  0.7302423603793466


### 3. SGDClassifier

In [21]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(max_iter=1000,tol=1e-3)

In [22]:
sgd.fit(X_train,y_train)

SGDClassifier()

In [23]:
print("F1_Score -> SGDClassifier: " ,sgd.score(X_test,y_test))

F1_Score -> SGDClassifier:  0.7105263157894737


### 4. Gaussian Naive Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB

In [25]:
gnb = GaussianNB().fit(X_train,y_train)

In [26]:
print("F1_Score -> GaussianNB: " ,gnb.score(X_test,y_test))

F1_Score -> GaussianNB:  0.6539473684210526


### 5. MultinomialNB

In [27]:
from sklearn.naive_bayes import MultinomialNB

In [28]:
#mnb = MultinomialNB().fit(X_train,y_train)

In [29]:
#print("F1_Score -> MultinomialNB: " ,mnb.score(X_test,y_test))

Note: Multinomial Naive Bayes gives an error because it is generally used to work on word count in given text document and count is always non-negative.

### 5. KNNClassifier with GridSearchCV

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

params_knn = {'n_neighbors':[2,4,5],'weights':['uniform','distance'],'metric':['euclidean', 'manhattan','minkowski']}
knn=GridSearchCV(KNeighborsClassifier(algorithm='auto'),param_grid=params_knn,cv=10)
knn.fit(X_train,y_train)


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': [2, 4, 5],
                         'weights': ['uniform', 'distance']})

In [31]:
print("F1_Score -> KNN CLassifier: " ,knn.score(X_test,y_test))

F1_Score -> KNN CLassifier:  0.65


### 6. MLP Classifier

In [32]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier(max_iter=1000).fit(X_train,y_train)

In [33]:
print("F1_Score -> MLPCLassifier: " ,mlpc.score(X_test,y_test))

F1_Score -> MLPCLassifier:  0.6802631578947368


### 7. LinearSVC

In [34]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC(max_iter=1000).fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [35]:
print("F1_Score -> Linear SVC: " ,lsvc.score(X_test,y_test))

F1_Score -> Linear SVC:  0.7276315789473684


### 8. SVC with GridSearchCV

In [36]:
from sklearn.svm import SVC

params_svc = {'kernel':['linear','poly','rbf','sigmoid'] , 'C':[0.1,0.5,1,10]}

#svc_gcv = GridSearchCV(SVC(),param_grid=params_svc,cv=10)
#svc_gcv.fit(X_train,y_train)


In [37]:
#svc_gcv.best_params_

In [38]:
#print("F1_Score -> SVC with GridSearchCV: " ,svc_gcv.score(X_test,y_test))

### 9. Decision Tree Classifier

In [39]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(criterion='entropy',splitter='best',max_depth=10)

dtc.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10)

In [40]:
print("F1_Score -> Decision Tree CLassifier: " ,dtc.score(X_test,y_test))

F1_Score -> Decision Tree CLassifier:  0.6815789473684211


### 10. Random Forest Classifier

In [41]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [42]:
print("F1_Score -> Random Forest CLassifier: " ,rfc.score(X_test,y_test))

F1_Score -> Random Forest CLassifier:  0.7368421052631579


### 11. Bagging Classifier

In [43]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(base_estimator= rfc) #Since SVC gave the best score till now
bag.fit(X_train,y_train)


BaggingClassifier(base_estimator=RandomForestClassifier())

In [44]:
print("F1_Score -> Bagging CLassifier: " ,bag.score(X_test,y_test))

F1_Score -> Bagging CLassifier:  0.7315789473684211


### 12. Using XG Boost

In [45]:
from xgboost import XGBRFClassifier

xgbr = XGBRFClassifier(max_depth=6,n_estimators=100,learning_rate=0.1)

xgbr.fit(X_train,y_train)

XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
                importance_type=None, interaction_constraints='',
                learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
                max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
                missing=nan, monotone_constraints='()', n_estimators=100,
                n_jobs=0, num_parallel_tree=100, objective='multi:softprob',
                predictor='auto', random_state=0, reg_alpha=0,
                sampling_method='uniform', ...)

In [46]:
print("F1_Score -> XGBRFCLassifier: " ,xgbr.score(X_test,y_test))

F1_Score -> XGBRFCLassifier:  0.7342105263157894


In [47]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=42,n_estimators=100,max_depth=6,learning_rate=0.1)

xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=42, reg_alpha=0, ...)

In [48]:
print("F1_Score -> XGB CLassifier: " ,xgb.score(X_test,y_test))

F1_Score -> XGB CLassifier:  0.7486842105263158


### 13. Bagging Combined with XG Boost

In [49]:
bag_xgb = BaggingClassifier(base_estimator= XGBClassifier(random_state=42,n_estimators=100,max_depth=6,learning_rate=0.1)) #Since SVC gave the best score till now
bag_xgb.fit(X_train,y_train)

BaggingClassifier(base_estimator=XGBClassifier(base_score=None, booster=None,
                                               callbacks=None,
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None,
                                               early_stopping_rounds=None,
                                               enable_categorical=False,
                                               eval_metric=None, gamma=None,
                                               gpu_id=None, grow_policy=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=0.1, max_bin=None,
                                               max_cat_to_onehot=None,
                                               m

In [50]:
print("F1_Score -> Bagging combined with XG Boost: " ,bag_xgb.score(X_test,y_test))

F1_Score -> Bagging combined with XG Boost:  0.7513157894736842


### 14. Adaboost Classifier

In [51]:
from sklearn.ensemble import AdaBoostClassifier
adab = AdaBoostClassifier(n_estimators=50,base_estimator=RandomForestClassifier()).fit(X_train,y_train)

In [52]:
print("F1_Score -> AdaboostClassifier with RFC as base estimator: " ,adab.score(X_test,y_test))

F1_Score -> AdaboostClassifier with RFC as base estimator:  0.7447368421052631


### 15. GradBoost Classifier


In [53]:
from sklearn.ensemble import  GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)

clf.score(X_test,y_test)

0.725

#### Since the best score is obtained by XGBoost Classifier we will use it on our training set

In [54]:
from xgboost import XGBClassifier

In [55]:
params_xgb = {'n_estimators' : [100,150,200] , 'max_depth': [5,6,10],'learning_rate':[0.1,0.2,0.3,0.5]}

In [56]:
xgb_gcv = GridSearchCV(XGBClassifier(random_state=42),param_grid=params_xgb,cv=5)

In [57]:
xgb2 = XGBClassifier(random_state=42,learning_rate=0.2, max_depth= 6, n_estimators=200)

In [58]:
xgb2.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=42, reg_alpha=0, ...)

In [59]:
xgb2.score(X_new,y)

0.9470495258166491

In [60]:
y_pred=rfc.predict(data_test_new)

In [61]:
import numpy as np
np.unique(y_pred,return_counts=True)

(array([0, 1, 2]), array([490, 999, 139]))

In [62]:
data_test['label']=y_pred
col1=data_test['id'].to_numpy()
col2=data_test['label'].to_numpy()
dataf=np.column_stack((col1,col2))

pd.DataFrame(dataf, columns=['id','label']).to_csv('submission8.csv',index=False)

In [63]:
import os

In [64]:
pwd

'/kaggle/working'

In [65]:
ls

__notebook__.ipynb  submission8.csv


In [66]:
from IPython.display import FileLink
FileLink(r'submission8.csv')

/kaggle/working/submission8.csv